<a href="https://colab.research.google.com/github/maciejskorski/ml_examples/blob/master/RenyiEntropyEstimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Renyi Entropy Estimation

This project implements the sample-optimal adaptive Renyi entropy estimator.